In [5]:
import os
import sys
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import io
from contextlib import redirect_stdout
from PIL import Image
from torch.utils.data import Subset, Dataset, random_split
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

# ----- Define a Tee class to write to multiple streams -----
class Tee(object):
    def __init__(self, *fileobjects):
        self.fileobjects = fileobjects

    def write(self, text):
        for f in self.fileobjects:
            f.write(text)
            f.flush()

    def flush(self):
        for f in self.fileobjects:
            f.flush()

# ----- Specify paths for saving logs, metrics, and models -----
log_file_path = "./logs/cifar10_training_log.txt"
metrics_file_path = "./metrics/cifar10_training_metrics.json"
best_model_path = "./models/cifar10_best_squeezenet"
save_path = "./models/cifar10_squeezenet"

# ----- Prepare the CIFAR-10 DataLoaders -----
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download and prepare datasets
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)

# Define the sizes for train and validation sets
train_size = int(0.8 * len(full_trainset))  # 80% for training
val_size = len(full_trainset) - train_size   # Remaining 20% for validation

# Split the dataset
trainset, valset = random_split(full_trainset, [train_size, val_size])

# Download and load the test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

batch_size = 256
num_workers = 2
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                       shuffle=False, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=num_workers)



# ----- Prepare the Model for CIFAR-10 -----
model = models.squeezenet1_1(weights=None)
# Modify the final layer for CIFAR-10 (10 classes)
model.classifier[1] = nn.Conv2d(512, 10, kernel_size=1)
model.num_classes = 10

# Add an AdaptiveAvgPool2d layer to ensure proper output size
model.classifier.add_module('flatten', nn.Flatten())

# ----- Setup Training Parameters -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 200
learning_rate = 0.001  # Reduced from 0.1 to 0.001
checkpoint_frequency = 10
start_epoch = 0
start_checkpoint = None

# If starting from a checkpoint, load the model state
if start_checkpoint:
    state_dict = torch.load(start_checkpoint, map_location=device)
    model.load_state_dict(state_dict)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Changed from SGD to Adam

# ----- Initialize Metrics Storage -----
metrics = {
    "epochs": [],
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}
best_val_acc = 0.0
best_epoch = None

# ----- Capture Standard Output During Training and Print to Console -----
log_capture = io.StringIO()
tee = Tee(sys.stdout, log_capture)  # This will print to stdout and also capture the output

# Add before training loop starts
writer = SummaryWriter(log_dir='./runs/cifar10_experiment')

with redirect_stdout(tee):
    print("Training SqueezeNet CIFAR-10 model...")
    for epoch in range(start_epoch, start_epoch + num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Save a checkpoint at the specified frequency
        if epoch % checkpoint_frequency == 0:
            torch.save(model.state_dict(), save_path + "_" + str(epoch) + ".pth")

        # Modified training loop with tqdm
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_epoch_loss = running_loss / total
        train_epoch_acc = 100. * correct / total

        # Evaluate on the validation set
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_epoch_loss = val_loss / val_total
        val_epoch_acc = 100. * val_correct / val_total

        # Store metrics for this epoch
        metrics["epochs"].append(epoch+1)  # Epochs are stored 1-indexed for clarity
        metrics["train_loss"].append(train_epoch_loss)
        metrics["train_acc"].append(train_epoch_acc)
        metrics["val_loss"].append(val_epoch_loss)
        metrics["val_acc"].append(val_epoch_acc)

        print(f"Epoch [{epoch+1}/{start_epoch + num_epochs}] - Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_acc:.2f}% | Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.2f}%")

        # Add TensorBoard logging after computing metrics
        writer.add_scalar('Loss/Train', train_epoch_loss, epoch)
        writer.add_scalar('Accuracy/Train', train_epoch_acc, epoch)
        writer.add_scalar('Loss/Validation', val_epoch_loss, epoch)
        writer.add_scalar('Accuracy/Validation', val_epoch_acc, epoch)

        # Save the best model based on validation accuracy
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_epoch = epoch+1
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model found at epoch {epoch+1} with Val Acc: {val_epoch_acc:.2f}%")
        
        # ----- Save metrics JSON after every epoch -----
        with open(metrics_file_path, "w") as f:
            json.dump(metrics, f, indent=4)
    
    print("Training complete.")
    torch.save(model.state_dict(), save_path + "_final.pth")

# Close the TensorBoard writer
writer.close()

# Write the captured logs to the specified log file (log_capture already contains the output)
with open(log_file_path, "w") as f:
    f.write(log_capture.getvalue())

# Optionally, print confirmation that the logs, metrics, and best model have been saved
print("Training log captured and saved to", log_file_path)
print("Training metrics saved to", metrics_file_path)
print("Best model saved from epoch", best_epoch, "with validation accuracy of", best_val_acc)

Files already downloaded and verified
Files already downloaded and verified
Training SqueezeNet CIFAR-10 model...


Epoch 1/200: 100%|██████████| 157/157 [00:03<00:00, 49.95it/s]


Epoch [1/200] - Train Loss: 1.9899, Train Acc: 22.96% | Val Loss: 1.7159, Val Acc: 33.15%
New best model found at epoch 1 with Val Acc: 33.15%


Epoch 2/200: 100%|██████████| 157/157 [00:02<00:00, 52.71it/s]


Epoch [2/200] - Train Loss: 1.6500, Train Acc: 36.43% | Val Loss: 1.5984, Val Acc: 37.88%
New best model found at epoch 2 with Val Acc: 37.88%


...

Epoch [198/200] - Train Loss: 0.0852, Train Acc: 97.33% | Val Loss: 2.4672, Val Acc: 64.06%


Epoch 199/200: 100%|██████████| 157/157 [00:03<00:00, 52.25it/s]


Epoch [199/200] - Train Loss: 0.0921, Train Acc: 97.08% | Val Loss: 2.4557, Val Acc: 63.07%


Epoch 200/200: 100%|██████████| 157/157 [00:02<00:00, 52.58it/s]


Epoch [200/200] - Train Loss: 0.1047, Train Acc: 96.71% | Val Loss: 2.3981, Val Acc: 64.04%
Training complete.
Training log captured and saved to ./logs/cifar10_training_log.txt
Training metrics saved to ./metrics/cifar10_training_metrics.json
Best model saved from epoch 27 with validation accuracy of 64.62
